# ADK Application Testing

This notebook demonstrates how to test an ADK (Agent Development Kit) application.
It covers both local and remote testing, both with Agent Engine and Cloud Run.

<img src="https://github.com/GoogleCloudPlatform/agent-starter-pack/blob/main/docs/images/adk_logo.png?raw=true" width="400">


## Install dependencies

### Import libraries

In [ ]:
import os
from uuid import uuid4

from vertexai import agent_engines
from vertexai.preview.reasoning_engines import AdkApp

from app.agent import root_agent
from app.agent_engine_app import AgentEngineApp

### Local Testing

You can import directly the AgentEngineApp class within your environment. 
To run the agent locally, follow these steps:
1. Make sure all required packages are installed in your environment
2. The recommended approach is to use the same virtual environment created by the 'uv' tool
3. You can set up this environment by running 'make install' from your agent's root directory
4. Then select this kernel (.venv folder in your project) in your Jupyter notebook to ensure all dependencies are available

In [4]:
async def run_async_query_and_get_results(
    a_engine: AdkApp, git_diff_content: str, remote: bool = False
):
    user_id = "test_user_async"
    session_id = str(uuid4())
    state = {"state": {"git_diff": git_diff_content}}
    events = []

    if not remote:
        await a_engine.async_create_session(
            user_id=user_id, session_id=session_id, state=state
        )

    agent_response_iterator = a_engine.async_stream_query(
        message="Review PR code changes according to your instructions.",
        user_id=user_id,
        session_id=None if remote else session_id,
    )

    async for event in agent_response_iterator:
        events.append(event)

    return events

In [5]:
def get_code_fixes_and_pentests(results):
    """
    Get the code fixes and pentests from the results.
    """
    fixed_code_patches = {}
    pen_tests = {}

    for entry in results:
        actions = entry.get("actions", {})
        state_delta = actions.get("state_delta", {})

        if "fixed_code_patches" in state_delta:
            fixed_code_patches = state_delta["fixed_code_patches"]
        if "pen_tests" in state_delta:
            pen_tests = state_delta["pen_tests"]

    return fixed_code_patches, pen_tests

In [ ]:
with open("../tests/samples/sample.diff") as f:
    git_diff = f.read()

agent_engine = AgentEngineApp(agent=root_agent)
results = await run_async_query_and_get_results(agent_engine, git_diff)
fixes, pentests = get_code_fixes_and_pentests(results)

In [ ]:
results

In [ ]:
fixes

In [ ]:
pentests

### Remote Testing

Using Agent Engine deployment to Vertex AI. Need to add REASONING_ENGINE_ID from your Vertex deployment.

In [30]:
AGENT_ENGINE_ID = f"projects/{os.environ.get('GOOGLE_CLOUD_PROJECT')}/locations/us-central1/reasoningEngines/2542396338859933696"
remote_agent_engine = agent_engines.get(AGENT_ENGINE_ID)

remote_results = await run_async_query_and_get_results(
    remote_agent_engine, git_diff, remote=True
)
r_fixes, r_pentests = get_code_fixes_and_pentests(remote_results)

In [ ]:
r_fixes

In [ ]:
r_pentests